<h1 align="center"><font color=#2980B9>Automatic Revenue Predictor</font></h1>

## <font color=#9B59B6>Table of Content</font>
1. Introduction
2. Pre-requisites
3. Data Set
4. The Solution
5. Conclusion

## <font color=#9B59B6>Introduction</font>
This is a small assignment to fulfil the technical requirements for _Data Engineer Role_ at **Snippy**.<br>
The Problem Statement requires us to create a Machine Learning Model that can predict revenue of a movie at the international box office.<br>
The Datasets that have been provided to us are "box_office_test.csv" and "box_office_train.csv". It is evident that we have to use one of them to train our model and the other will be used to test the accuracy of this model.<br>
Lets go!

## <font color=#9B59B6>Pre-requisites</font>
1. Pandas
2. NumPy
3. Scikit Learn
4. eli5
5. TFID
6. LGB boost
7. GB regressor
8. SciPy
9. Plotly

### Importing all the libraries and packages
Below is the code where I have imported all the required packages and libraries.

In [46]:
import os, datetime, json, sklearn, eli5, time, ast, pandas as pd, numpy as np, plotly.offline as py, plotly.graph_objs as go
import plotly.tools as tls, matplotlib.pyplot as plt, seaborn as sns, xgboost as xgb, lightgbm as lgb

from matplotlib import pyplot
from PIL import Image
from urllib.request import urlopen
from collections import Counter
from nltk.util import ngrams
from scipy import stats
from scipy.sparse import hstack, csr_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.model_selection import (
    train_test_split,
    KFold,
    cross_val_score
)

from sklearn.feature_extraction.text import (
    TfidfVectorizer,
    CountVectorizer
)

from sklearn.preprocessing import (
    StandardScaler,
    LabelEncoder
)

from sklearn.metrics import (
    accuracy_score,
    mean_squared_error,
    r2_score
)

from sklearn.linear_model import (
    LinearRegression,
    LogisticRegression
)

print('All libraries were imported successfully')

All libraries were imported successfully


Setting some other options for pandas, pyplot, and plotly


In [47]:
pd.set_option('max_columns', None) # sets no limit for columns that are displayed
plt.style.use('ggplot') # using ggplot as default for pyplot
py.init_notebook_mode(connected=True) # initializing notebook mode for plotly
print ('Default modes set successfully')


Default modes set successfully


## <font color=#9B59B6>Analyzing The Dataset</font>
Reading the dataset that has to be used to train our model into a Pandas Dataframe.<br>
We also want to have a look at the entries that are there in the dataset.<br>
This can be achieved using df.head() method which prints first 5 entries by default.<br>

In [48]:
df = pd.read_csv("../data/box_office_train.csv") # Creates Pandas Dataframe from CSV files
df.head() # Prints the first 5 entries in our dataset

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,0,NaN,3000000,"[{'id': 37, 'name': 'Western'}]",NaN,tt0040724,en,Red River,Dunson is driving his cattle to Red River when...,6.090180,/9TwFtppqU0zUvtNTIDxkV7zQebC.jpg,"[{'name': 'United Artists', 'id': 60}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",8/26/48,133.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Big as the men who faced this challenge! Bold ...,Red River,"[{'id': 1556, 'name': 'texas'}, {'id': 3851, '...","[{'cast_id': 10, 'character': 'Thomas Dunson',...","[{'credit_id': '52fe4384c3a36847f805a485', 'de...",9012000
1,1,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,tt0099028,en,American Dream,Chronicles the six-month strike at Hormel in A...,0.075885,/oi2vKZROaDoJpt1vvKXeQOTKlNK.jpg,NaN,"[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",9/28/90,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"The award-winning film of American lives, Amer...",American Dream,"[{'id': 187056, 'name': 'woman director'}]","[{'cast_id': 5, 'character': 'Himself', 'credi...","[{'credit_id': '52fe45c2c3a36847f80d8e9d', 'de...",269823
2,2,"[{'id': 416491, 'name': 'Major Korenev Trilogy...",5000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,tt0325005,ru,Антикиллер,"Former criminal investigator, Major Korenev, n...",0.191539,/pkdnzpmacniXKJnEmlD7q5KCFrw.jpg,NaN,"[{'iso_3166_1': 'RU', 'name': 'Russia'}]",8/1/02,113.0,"[{'iso_639_1': 'ru', 'name': 'Pусский'}]",Released,NaN,Antikiller,NaN,"[{'cast_id': 6, 'character': 'Cross (Krest)', ...","[{'credit_id': '52fe45e5c3a36847f80e08db', 'de...",340312
3,3,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,tt0101595,en,Close My Eyes,"After some years of tension, Richard begins a ...",1.404434,/r6llY1uyhQOXLh8lFuohoed315.jpg,"[{'name': 'Film 4', 'id': 2335}, {'name': 'Fil...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",11/22/91,109.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,There are some relationships so taboo they're ...,Close My Eyes,"[{'id': 255, 'name': 'male nudity'}, {'id': 29...","[{'cast_id': 1, 'character': 'Sinclair', 'cred...","[{'credit_id': '52fe4881c3a36847f816abb5', 'de...",135893
4,4,"[{'id': 112636, 'name': 'The Blues Brothers Co...",28000000,"[{'id': 10402, 'name': 'Music'}, {'id': 28, 'n...",NaN,tt0118747,en,Blues Brothers 2000,"Elwood, the now lone ""Blues Brother"" finally r...",13.162168,/87FRtfvpH6nGhGRIDymcNGn6duK.jpg,"[{'name': 'Universal Pictures', 'id': 33}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2/5/98,123.0,"[{'iso_639_1': 'ru', 'name': 'Pусский'}, {'iso...",Released,NaN,Blues Brothers 2000,"[{'id': 378, 'name': 'prison'}, {'id': 2636, '...","[{'cast_id': 16, 'character': 'Elwood Blues', ...","[{'credit_id': '52fe445d9251416c750321a7', 'de...",14051384


Now let's find out how many rows, columns and entries our dataset has.

In [49]:
df.info() # prints valuable info about the dataframe
print(f"The no. of rows x columns is:",df.shape) # prints the number of rows and columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     2400 non-null   int64  
 1   belongs_to_collection  486 non-null    object 
 2   budget                 2400 non-null   int64  
 3   genres                 2394 non-null   object 
 4   homepage               765 non-null    object 
 5   imdb_id                2400 non-null   object 
 6   original_language      2400 non-null   object 
 7   original_title         2400 non-null   object 
 8   overview               2393 non-null   object 
 9   popularity             2400 non-null   float64
 10  poster_path            2399 non-null   object 
 11  production_companies   2276 non-null   object 
 12  production_countries   2357 non-null   object 
 13  release_date           2400 non-null   object 
 14  runtime                2398 non-null   float64
 15  spok

We can see that the total number of rows is 2400. However, in many of the columns we can see the number of objects is less than 2400. This must mean that many entries are null.<br>
We have to find out how many such null entries are present in our dataset, If not properly handled, they can create issues in the future when we train our model.<br>
Duplicated data is also another issue that we need to handle here. 

In [50]:
print(df.isnull().sum()) # Prints a count of all the entries that are null in every column in this dataframe
print("The total number of duplicated objects in this dataset is:",df.duplicated().sum()) # Prints a count of all the entries that are duplicated
print("The total number of values that are null in this dataset is:",df.isnull().sum().sum()) 

id                          0
belongs_to_collection    1914
budget                      0
genres                      6
homepage                 1635
imdb_id                     0
original_language           0
original_title              0
overview                    7
popularity                  0
poster_path                 1
production_companies      124
production_countries       43
release_date                0
runtime                     2
spoken_languages           14
status                      0
tagline                   477
title                       0
Keywords                  226
cast                       10
crew                       13
revenue                     0
dtype: int64
The total number of duplicated objects in this dataset is: 0
The total number of values that are null in this dataset is: 4472
